In [4]:
import json, os

testydata: str = "./../../../testymolo/media/data/"

Proteins:list = json.load(open(os.path.join(testydata,'Protein.temp.json')))
data_ac:int = Proteins[0]['data_ac']
PP = [ pp for pp in Proteins if (pp['isPP'] and (pp['data_ac'] == data_ac)) ][0]
nsps = list([ nsp for nsp in Proteins if (nsp['derivedFromPP'] and (nsp['data_ac'] == data_ac)) ])

Subseqs:list = json.load(open(os.path.join(testydata,'Subseq.temp.json')))
Moduli:list = json.load(open(os.path.join(testydata,'Modulo.temp.json')))
Profiles:list = json.load(open(os.path.join(testydata,'Profile.temp.json')))
PolyProtein:list = json.load(open(os.path.join(testydata,'PolyProtein.temp.json')))

protein_id:list = [PP['id']] + [nsp['id'] for nsp in nsps]

subseq:list = []
modulo:list= []
profile:list = []
for sseq in Subseqs:
    if sseq['origin'] in protein_id:
        subseq.append(sseq)
        
        for prof in Profiles:
            if prof['id'] == sseq['profile']:
                profile.append(prof)

                for mod in Moduli:
                    if mod['id'] == prof['modulo']:
                        modulo.append(mod)

# 1er PP
# incluants ses nsps 
# tous les subseqs correspondant
# tous les profiles correspondant
# tous les modules correspondant

In [5]:
print(PP)
print(nsps[0])
print(subseq[-1])
print(profile[0])
print(modulo[-1])

{'id': 1, 'isPP': True, 'derivedFromPP': False, 'genbank': 'NP_066134.1', 'name': 'ORF1ab', 'data_ac': 473, 'complete': True, 'organism': 11120, 'header': 'ORF1ab_polyprotein__(@473)', 'sequence': 'MASSLKQGVSPKPRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEE

# Figure 

In [6]:
def generate_figure():
    ## generate figure
    real_WIDTH = 1860
    WIDTH = 1800
    x0 = 30  # left offset
    html:str = ""
    chtext:bool = False
    chlvl:int = 0
    y_top = 50
    y_height = 20

    #organism = protein.organism
    L = len(PP['sequence'])
    
    #html += "<div class='button-container'>"
    #html += "<button class='compare' >Compare with <span class='uni_star' value=false style='color: orange;'>★</span></button>\n"
    #html += "<button class='minus' onclick='updateSection_figure_minus()'>-</button></div>\n"
    html += f"<span class='sequence_selected_icon' arrow='arrow-thick-right' ><span class='whitespace'>_</span></span><span class='header'>{PP['header']}</span> \n"
    html += f"<div class='svg-container'><svg protein={PP['id']} width='{str(real_WIDTH)}' height='300'> \n"
    html += f"<rect class='background' width='100%' height='100%' fill='white' fill-opacity='1' stroke='black' /> \n"
    html += f"<rect class='protein' x='{x0}' y='{y_top}%' width='{WIDTH}' height='{y_height}%' fill='blue' fill-opacity='0.1'/> \n"
    text1 = f"name:{'###'},    polyprotein:{PP['isPP']},  derived from polyprotein:{PP['derivedFromPP']}"
    text2 = f"organism:{'###'},    acc:{PP['data_ac']},    length:{L}"
    html += f"<text x='10' y='5%' >{text1}</text>\n"
    html += f"<text x='10' y='15%' >{text2}</text>\n"

    i=0
    #subseq = data.Subseq.objects.filter(origin=protein)
    for sseq in subseq:
        i+=1
        lenseq = len(PP['sequence'][sseq['start']-1:sseq['end']])
        w = ((lenseq +1) / L) * WIDTH
        x = (sseq['start']-1)/L *WIDTH +x0
        # find module
        if sseq['profile'] != 'null':
            prof = [prof for prof in profile if prof['id'] == sseq['profile']][0]
            mod = [mod for mod in modulo if mod['id'] == prof['modulo']][0]
        else:
            mod = "?"
        
        html += f"<rect class='subseq' subseq='{sseq['id']}' height='{y_height}%' y='{y_top}%' fill='blue' fill-opacity='0.2' "
        html += f"x='{x}' width='{w}' />\n"
        if(w < (0.020 *WIDTH)):
            chtext = True
            chlvl += 1  
        else:
            chlvl = chlvl-1 if(chlvl > 0) else 0

        #text module_name
        if(w > (0.020 *WIDTH)):
            x += w*2/5
        y = y_top-5
        if(chtext):
            y -= (5 *chlvl)
        html += f"<text class='module_name' id='module_name_{i}' font-size='14' y='{y}%' "
        html += f"x='{x}' >{mod['id']}</text>\n"

        #line separator
        x1 = ((sseq['end']) / L) *WIDTH +x0
        html += f"<line class='separator' id='separator_{i}' y1='{y_top}%' y2='{y_top+y_height}%' stroke='black' stroke-width='2' "
        html += f"x1='{x1}' x2='{x1}' />\n"

        #text numbering
        if(i < len(subseq)):
            y = y_top + y_height + 5
            if(chtext):
                chtext = False
                y += (5 *chlvl)
            x = ((sseq['end']) / L) *WIDTH +x0
        else:
            text_length:int = len(str(sseq['end']))
            y = y_top + y_height + 10 + 5 *chlvl
            x = (real_WIDTH - text_length *10)
        html += f"<text class='numbering' id='numbering_{i}' font-size='10'  "
        html += f" y='{y}%' x='{x}' >{sseq['end']}</text>\n"
    html += "</svg></div>"

    return "<div class='figure'>"+html+"</div>"

from IPython.core.display import display, HTML
display(HTML(generate_figure()))

In [7]:
# (0,0) 
#  --------------------------------> 
# |
# |
# |
# |
# |
# |
# |
# |
# |
# |
# |
#\/

In [8]:
### META ###
#-1.1 factoriser : { element html <-> data...truc }                                                                :classic:
#-1.2 pre-calculer : coordonnées relatives (dépendance entre element html) et absolue (réalité sur la figure)      :chiant:
#                 -> SUM(relative coord)[residu] * coef[residu->pixel] + offset[pixel] => absolute coord[pixel] 
#-2. template-r : template-ception ? { template <-> data...truc }                                                  :sexy:


In [9]:
#:classic: 
def formating_inputdata():
    
    #data:list = []
    ##META## 
    # if len(data) == 1 : une protein
    # if len(data) > 1 : 1er->
        # -> proteins :dict
            # \-> { **prot, 'subseq':[{ **sseq, 'modulo':mod }] }
    import json, os
    testydata: str = "./../../../testymolo/media/data/"
    PolyProtein:list = json.load(open(os.path.join(testydata,'PolyProtein.temp.json')))

    data:list = [ 
            { **PP , 'start':1 },
            *[ { **nsp } for nsp in nsps ],
        ]

    for i in range(len(data)):
        data[i]['subseq'] = []

        for item in PolyProtein:
            if item['protein'] == data[i]['id']:
                data[i]['start'] = item['start']
                break

        for sseq in subseq:
            if sseq['origin'] == data[i]['id']:
                
                for prof in profile:
                    if prof['id'] == sseq['profile']:

                        for mod in modulo:
                            if mod['id'] == prof['modulo']:
                                
                                data[i]['subseq'].append({ **sseq , 'modulo':mod })
                                break
                        break
    return data

for item in formating_inputdata():
    print(item)

{'id': 1, 'isPP': True, 'derivedFromPP': False, 'genbank': 'NP_066134.1', 'name': 'ORF1ab', 'data_ac': 473, 'complete': True, 'organism': 11120, 'header': 'ORF1ab_polyprotein__(@473)', 'sequence': 'MASSLKQGVSPKPRDVILVSKDIPEQLCDALFFYTSHNPKDYADAFAVRQKFDRSLQTGKQFKFETVCGLFLLKGVDKITPGVPAKVLKATSKLADLEDIFGVSPLARKYRELLKTACQWSLTVEALDVRAQTLDEIFDPTEILWLQVAAKIHVSSMAMRRLVGEVTAKVMDALGSNLSALFQIVKQQIARIFQKALAIFENVNELPQRIAALKMAFAKCARSITVVVVERTLVVKEFAGTCLASINGAVAKFFEELPNGFMGSKIFTTLAFFKEAAVRVVENIPNAPRGTKGFEVVGNAKGTQVVVRGMRNDLTLLDQKADIPVEPEGWSAILDGHLCYVFRSGDRFYAAPLSGNFALSDVHCCERVVCLSDGVTPEINDGLILAAIYSSFSVSELVTALKKGEPFKFLGHKFVYAKDAAVSFTLAKAATIADVLRLFQSARVIAEDVWSSFTEKSFEFWKLAYGKVRNLEEFVKTYVCKAQMSIVILAAVLGEDIWHLVSQVIYKLGVLFTKVVDFCDKHWKGFCVQLKRAKLIVTETFCVLKGVAQHCFQLLLDAIHSLYKSFKKCALGRIHGDLLFWKGGVHKIVQDGDEIWFDAIDSVDVEDLGVVQEKSIDFEVCDDVTLPENQPGHMVQIEDDGKNYMFFRFKKDENIYYTPMSQLGAINVVCKAGGKTVTFGETTVQEIPPPDVVPIKVSIECCGEPWNTIFKKAYKEPIEVDTDLTVEQLLSVIYEKMCDDLKLFPEAPEPPPFENVALVDKNGKDLDCIKSCHLIYRDYESDDDIEEEDAEECDTDSGEAEE

In [30]:
#:chiant:

from IPython.core.display import display, HTML

class Figure():

    Templates:list = {'protein':''}
    TRUE_WIDTH:int = 1860  # [pixel]
    WIDTH:int = 1780  # [pixel]
    x0:int = 12  # x_offset  [pixel]
    y0 = 75  # y_offset  [pixel]
    h = 60  # [pixel]
    HTML:str = ""

    def __init__(self, data:list) -> None:
        """ return HTML content """
        self.data = data
        self.PP = data[0]
        #self.HEIGHT = 300  # [pixel] 
        N:int = len([p for p in self.data if len(p['subseq']) > 0])
        print("nbr of protein:", N)
        self.HEIGHT = Figure.y0 + (2 if len(data) >1 else 1) *2*Figure.h  # [pixel] 

        self.HTML += f"<span class='sequence_selected_icon' arrow='arrow-thick-right' ><span class='whitespace'>_</span></span><span class='header'>{self.PP['header']}</span> \n"
        self.HTML += f"<div class='svg-container'><svg protein={self.PP['id']} width='{str(Figure.TRUE_WIDTH)}' height='{self.HEIGHT}'> \n"
        self.HTML += f"<rect class='background' width='100%' height='100%' fill='white' fill-opacity='1' stroke='black' /> \n"
    
        self.HTML += self.Protein(0) 
        for protein_i in range(1, len(self.data)):
            if(len(self.data[protein_i]['subseq']) > 0):
                self.HTML += self.Protein(protein_i, 1) 

    def diplay(self):
        return display(HTML(self.HTML))
    
    def Protein(self, i:int, n:int=0) -> str:
        protein = self.data[i]
        html:str = ""

        xi = Figure.x0  # x_offset 
        xi += ((protein['start']-1) / self.PP['length']) * Figure.WIDTH  # + relative position from PP
        yi = Figure.y0 + (n)*2*Figure.h   # as downwards as many proteins to display
        wi = (protein['length'] / self.PP['length']) * Figure.WIDTH  # adjust width with length protein
        hi = Figure.h

        #dotted line for nsps
        if n > 0:
            x_bl1 = xi
            y_bl1 = Figure.x0 -30
            x_bl2 = x_bl1
            y_bl2 = yi + hi + 30
            html += f"<line class='borderline' id='borderline_{i}' x1='{x_bl1}' y1='{y_bl1}' x2='{x_bl2}' y2='{y_bl2}' stroke='grey' stroke-width='1' stroke-dasharray='4' />\n"

        #protein name
        x_txt = xi +5
        y_txt = yi -20
        html += f"<text class='module_name' id='module_name_{protein['name']}' fill='grey' font-weight='bold' font-size='14' x='{x_txt}' y='{y_txt}'>{protein['name']}</text>\n"

        #protein rect
        html += f"<rect class='protein' x='{xi}' y='{yi}' width='{wi}' height='{hi}' fill='blue' fill-opacity='0.1'/> \n"

        chtext:bool = False
        chlvl:int = 0

        for sseq_i in range(len(protein['subseq'])):
            html += self.Subseq(protein, sseq_i, xi, yi, wi, hi, chtext, chlvl)
            #pass
            
        return html

    def Subseq(self, protein, subseq_i, *coord) -> str:
        subseq = protein['subseq'][subseq_i]
        xi, yi, wi, hi, chtext, chlvl, *_ = coord
        html:str = ""

        L = len(protein['sequence'])
        len_sseq:int = len(protein['sequence'][subseq['start']-1:subseq['end']])

        xj = (subseq['start']-1)/L *wi + xi
        yj = yi
        wj = ((len_sseq +1) / L) *wi
        hj = hi

        html += f"<rect class='subseq' subseq='{subseq['id']}' x='{xj}' y='{yj}' width='{wj}' height='{hj}' fill='green' fill-opacity='0.2'  />\n"
    
        if(wj < (0.022 *Figure.WIDTH)):
            chtext = True
            chlvl += 1  
        else:
            chlvl = chlvl-1 if(chlvl > 0) else 0

        # text module_name
        x_mod = xj
        y_mod = yj
        if(wj > (0.022 *Figure.WIDTH)):
            x_mod += wj*2/5
        y_mod -= 5
        if(chtext):
            y_mod -= (12 *chlvl)
        html += f"<text class='module_name' id='module_name_{subseq['modulo']['id']}' font-size='12' x='{x_mod}' y='{y_mod}'>{subseq['modulo']['id']}</text>\n"

        #line separator
        x_sep1 = (subseq['end'] / L) *wi +xi 
        y_sep1 = yi
        x_sep2 = x_sep1
        y_sep2 = y_sep1 + Figure.h
        html += f"<line class='separator' id='separator_{subseq_i}' x1='{x_sep1}' y1='{y_sep1}' x2='{x_sep2}' y2='{y_sep2}' stroke='black' stroke-width='2' />\n"


        #text numbering
        x_numb = x_sep1
        y_numb = yi + Figure.h + 15
        if(chtext):
            chtext = False
            y_numb += (12 *chlvl)
        html += f"<text class='numbering' id='numbering_{subseq_i}' font-size='10' x='{x_numb}' y='{y_numb}'>{subseq['end']}</text>\n"

        return html


    


In [31]:
fig0 = Figure(formating_inputdata())
fig0.diplay()


nbr of protein: 4


In [ ]:
def Main():
    ## generate figure
    real_WIDTH = 1860
    WIDTH = 1800
    x0 = 30  # left offset
    html:str = ""
    chtext:bool = False
    chlvl:int = 0
    y_top = 50
    y_height = 20

    #organism = protein.organism
    L = len(PP['sequence'])
    
    #html += "<div class='button-container'>"
    #html += "<button class='compare' >Compare with <span class='uni_star' value=false style='color: orange;'>★</span></button>\n"
    #html += "<button class='minus' onclick='updateSection_figure_minus()'>-</button></div>\n"
    html += f"<span class='sequence_selected_icon' arrow='arrow-thick-right' ><span class='whitespace'>_</span></span><span class='header'>{PP['header']}</span> \n"
    html += f"<div class='svg-container'><svg protein={PP['id']} width='{str(real_WIDTH)}' height='600'> \n"
    html += f"<rect class='background' width='100%' height='100%' fill='white' fill-opacity='1' stroke='black' /> \n"
    html += f"<rect class='protein' x='{x0}' y='{5}' width='{WIDTH}' height='{y_height}%' fill='blue' fill-opacity='0.1'/> \n"
    #text1 = f"name:{'###'},    polyprotein:{PP['isPP']},  derived from polyprotein:{PP['derivedFromPP']}"
    #text2 = f"organism:{'###'},    acc:{PP['data_ac']},    length:{L}"
    #html += f"<text x='10' y='5%' >{text1}</text>\n"
    #html += f"<text x='10' y='15%' >{text2}</text>\n"

    i=0
    #subseq = data.Subseq.objects.filter(origin=protein)
    for sseq in subseq:
        i+=1
        lenseq = len(PP['sequence'][sseq['start']-1:sseq['end']])
        w = ((lenseq +1) / L) * WIDTH
        x = (sseq['start']-1)/L *WIDTH +x0
        # find module
        if sseq['profile'] != 'null':
            prof = [prof for prof in profile if prof['id'] == sseq['profile']][0]
            mod = [mod for mod in modulo if mod['id'] == prof['modulo']][0]
        else:
            mod = "?"
        
        html += f"<rect class='subseq' subseq='{sseq['id']}' height='{y_height}%' y='{y_top}%' fill='blue' fill-opacity='0.2' "
        html += f"x='{x}' width='{w}' />\n"
        if(w < (0.020 *WIDTH)):
            chtext = True
            chlvl += 1  
        else:
            chlvl = chlvl-1 if(chlvl > 0) else 0

        #text module_name
        if(w > (0.020 *WIDTH)):
            x += w*2/5
        y = y_top-5
        if(chtext):
            y -= (5 *chlvl)
        html += f"<text class='module_name' id='module_name_{i}' font-size='14' y='{y}%' "
        html += f"x='{x}' >{mod['id']}</text>\n"

        #line separator
        x1 = ((sseq['end']) / L) *WIDTH +x0
        html += f"<line class='separator' id='separator_{i}' y1='{y_top}%' y2='{y_top+y_height}%' stroke='black' stroke-width='2' "
        html += f"x1='{x1}' x2='{x1}' />\n"

        #text numbering
        if(i < len(subseq)):
            y = y_top + y_height + 5
            if(chtext):
                chtext = False
                y += (5 *chlvl)
            x = ((sseq['end']) / L) *WIDTH +x0
        else:
            text_length:int = len(str(sseq['end']))
            y = y_top + y_height + 10 + 5 *chlvl
            x = (real_WIDTH - text_length *10)
        html += f"<text class='numbering' id='numbering_{i}' font-size='10'  "
        html += f" y='{y}%' x='{x}' >{sseq['end']}</text>\n"
    html += "</svg></div>"

    return "<div class='figure'>"+html+"</div>"

from IPython.core.display import display, HTML
display(HTML(Main()))